In [1]:
# ============================================================
# Train ML Models for Solar Panel Monitor
# ============================================================

# 1. Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor
import joblib

# ------------------------------------------------------------
# 2. Load preprocessed dataset
# ------------------------------------------------------------
df = pd.read_csv("../data/solar_panel_monitor_dataset.csv")
print("Dataset shape:", df.shape)
df.head()

# ------------------------------------------------------------
# 3. Select features and targets
# ------------------------------------------------------------
# Features (choose based on your preprocessing)
features = [
    "ldr_tl","ldr_tr","ldr_bl","ldr_br",
    "ldr_sum","ldr_left","ldr_right","ldr_top","ldr_bottom",
    "sin_hour","cos_hour","sin_min","cos_min",
    "sun_azimuth","sun_elevation"
]

X = df[features]

# Targets (direct angles or deltas)
y_h = df["target_horiz"] if "target_horiz" in df else df["delta_horiz"]
y_v = df["target_vert"] if "target_vert" in df else df["delta_vert"]

# ------------------------------------------------------------
# 4. Split into train/test sets
# ------------------------------------------------------------
X_train, X_test, y_h_train, y_h_test = train_test_split(X, y_h, test_size=0.2, shuffle=False)
_, _, y_v_train, y_v_test = train_test_split(X, y_v, test_size=0.2, shuffle=False)

print("Train samples:", X_train.shape[0], " | Test samples:", X_test.shape[0])

# ------------------------------------------------------------
# 5. Train models
# ------------------------------------------------------------
# Horizontal angle model
model_h = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_h.fit(X_train, y_h_train)

# Vertical angle model
model_v = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_v.fit(X_train, y_v_train)

# ------------------------------------------------------------
# 6. Evaluate models
# ------------------------------------------------------------
pred_h = model_h.predict(X_test)
pred_v = model_v.predict(X_test)

print("Horizontal Angle MAE:", mean_absolute_error(y_h_test, pred_h))
print("Vertical Angle MAE   :", mean_absolute_error(y_v_test, pred_v))
print("Horizontal R²:", r2_score(y_h_test, pred_h))
print("Vertical R²   :", r2_score(y_v_test, pred_v))

# ------------------------------------------------------------
# 7. Save models
# ------------------------------------------------------------
joblib.dump(model_h, "model_h.joblib")
joblib.dump(model_v, "model_v.joblib")

print("✅ Models saved: model_h.joblib & model_v.joblib")


ModuleNotFoundError: No module named 'pandas'